## Project for rmrb - Part #[1]

### Corpus construction, text cleaning and filtering

In [10]:
import dask
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import jieba
import re
import string
from zhon.hanzi import punctuation, stops, non_stops
import gensim
import timeit
import warnings
import zipfile
import os
import sys
warnings.filterwarnings('ignore')

In [11]:
# need to unpack the 7z files before proceeding
user = input("Who is using the notebook? ")
if user == "Tim":
    corpus_name = "/Users/timqzhang/Desktop/UChicago/MACSS_Spring_2020/content_local/rmrb/7z"
elif user == "Linghui":
    corpus_name = "/Users/linghuiwu/uchicago/courseworks/soci40133/rmrb/7z"
elif user == "Minghao":
    corpus_name = "D:/学习资料/研一三学期/Content Analysis/Final_Project/rmrb/7z"

Who is using the notebook? Tim


In [17]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 16


In [29]:
# functions that used to tokenize and normalize sentences

def cut_sent(para): 
    para = re.sub('[\"\']', '', para)
    para = re.sub('([｡。.！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)
    para = re.sub('([。！？\?\…\.]+[”’])([^，.。！？\?])', r'\1\n\2', para)
    para = para.rstrip()
    return para.split("\n")


def clean_sent(sent): 
    remove = str.maketrans('○〇', '零零', string.punctuation) 
    sent = sent.translate(remove)
    sent = re.sub('[{}]+'.format(punctuation + non_stops), '', sent)
    return sent.strip()


def tokenize_sent(sent, min_token_length=2): 
    line = [token for token in jieba.cut(clean_sent(sent), use_paddle=False) if token != ' ' and len(token) >= min_token_length]
    return line


def join_para_tokens(para_tokens): 
    return '\n'.join(list(dask.compute(*para_tokens)))


def tokenize_para(text): 
    para_tokens = []
    for para in text:
        for sent in cut_sent(para):
            tokenized_sent = dask.delayed(tokenize_sent(sent))
#            tokenized_sent = tokenize_sent(sent)
            para_tokens.append(tokenized_sent)
    return list(dask.compute(*para_tokens))
#    return para_tokens
    

# def write_tokens_to_file(file_tokens): 
#     with open(tokens_path, 'w') as fh_w: 
#         for para_tokens in file_tokens: 
#             fh_w.write(para_tokens)
#             fh_w.write('\n')

In [30]:
# a function used to drop unnecessary information in texts

def text_cleaning(lists, text):

    clean_text = [re.sub('\n', '', x).strip() for x in text if x != '\n']
    title, authors = lists

    # clean overlap title
    for title_piece in title:
        for sent in clean_text[:10]:
            if title_piece in sent and len(title_piece) + 1 > len(sent):
                clean_text[clean_text.index(sent)] = sent[
                    sent.index(title_piece) + len(title_piece):]
            elif sent in title_piece:
                clean_text[clean_text.index(sent)] = ''  
    # if after cleaning the list goes empty, then return False to show this
    # news is trivial (same below)
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # clean overlap authors at the beginning
    for author_piece in authors:
        if author_piece in clean_text[0]:
            clean_text[0] = clean_text[0][clean_text[0].index(
                author_piece) + len(author_piece) + 1:]
    
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # clean overlap authors at the end
    for author_piece in authors:
        if author_piece in clean_text[-1]:
            ind = clean_text[-1].rfind('.')
            if ind == -1:
                clean_text = clean_text[:-1]
                break
            else:
                clean_text[-1] = clean_text[-1][:ind + 1]
    
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning ‘【】’, like【新华社电讯】, etc. at the beginning
    if '【' in clean_text[0][:20]:
        clean_text[0] = clean_text[0][clean_text[0].index('】') + 1:]
        
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant parathesis at the beginning
    if '（' in clean_text[0][:25] and len(clean_text) > 1:
        if '）' in clean_text[0]:
            clean_text[0] = clean_text[0][clean_text[0].index('）') + 1:]
        elif '）' in clean_text[1] and '（' not in clean_text[0]:
            clean_text = clean_text[1:]
            clean_text[0] = clean_text[0][clean_text[0].index('）') + 1:]
            
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant colons at the beginning like “新华社报讯:”
    if '：' in clean_text[0][:20]:
        clean_text[0] = clean_text[0][clean_text[0].index('：') + 1:]
        
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant info at the beginning like “新华社电”
    if '新华社' in clean_text[0][:50]:
        if '电' in clean_text[0][:50]:
            clean_text[0] = clean_text[0][clean_text[0].index('电') + 1:]
        elif '讯' in clean_text[0][:50]:
            clean_text[0] = clean_text[0][clean_text[0].index('讯') + 1:]
            
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant info at the beginning like “本报讯”
    if '本报讯' in clean_text[0][:50]:
        clean_text[0] = clean_text[0][clean_text[0].index('讯') + 1:]
        
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant info at the end like “新华社稿”
    if '（' in clean_text[-1][-15:]:
        clean_text[-1] = clean_text[-1][:clean_text[-1].index('（')]
        
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # cleaning the irrelevant info at the end like “新华社摄”
    if '摄' in clean_text[-1][-20:] and '记者' in clean_text[-1][-20:]:
        ind = clean_text[-1].rfind('.')
        if ind == -1:
            clean_text = clean_text[:-1]
        else:
            clean_text[-1] = clean_text[-1][:ind + 1]
    
    clean_text = list(filter(lambda x: x != '', clean_text))

    if len(clean_text) == 0:
        return clean_text, False

    # return the cleaned texts
    return clean_text, True

In [31]:
# define the main func for corpus load and processing

def loadcorpus_plus_cleaning(corpus_name):

    texts_raw = {'year': [], 'month': [], 'date': [],
                 'issue': [], 'column': [], 'title': [],
                 'author': [], 'text':[], 'tokenized_sent':[], 'token_para':[]}

    count = 0
    folders = sorted(os.listdir(corpus_name + "/"))
    
    # filter the year to 1965-1995
    year_list=[str(x) for x in range(1965,1996)]
    folders_filter=sorted([x for x in folders if x[:4] in year_list])
 
    for folder in folders_filter:
        if '7z' not in folder:
            count += 1
            print('Current running on {}, total: {}/372'.format(folder,
                                                                count), end='\r', flush=True)


            for file in os.listdir(corpus_name + "/" + folder + '/'):
                if 'md' in file:

                    # load in raw text

                    news_piece = []
                    news_loc = corpus_name + "/" + folder + '/'
                    f = open(news_loc + file, encoding='utf-8')
                    for line in f:
                        news_piece.append(line)
                    f.close()

                    # fill the info this news piece with cleaning

                    # issue (第x版) (only keep issue 1 - 4)
                    # some issues have extended infomation, just keep them here
                    issue_raw = re.sub('\n', '', news_piece[3])
                    if '第版' in issue_raw or int(re.search(r'\d+', issue_raw).group(0)) > 4:
                        continue
                    else:
                        if issue_raw.index(')') - issue_raw.index('(') == 1:
                            texts_raw['issue'].append(
                                str(re.search(r'\d+', issue_raw).group(0)))
                        else:
                            temp = issue_raw.index('(')
                            texts_raw['issue'].append(str(re.search(r'\d+', issue_raw).group(0)) + 
                                                      '_' + issue_raw[temp + 1:-1])

                    # title
                    clean_title_list = re.sub(
                        '\n', '', news_piece[0])[4:].split()
                    clean_title_list = list(
                        filter(lambda x: x != '', clean_title_list))
                    clean_title_str = ' '.join(clean_title_list)
                    # delete picture news
                    if '（图片）' in clean_title_str:
                        del texts_raw['issue'][-1]
                        continue
                    else:
                        texts_raw['title'].append(clean_title_str)
                    
                    # dates
                    texts_raw['year'].append(str(folder)[:4])
                    texts_raw['month'].append(str(folder)[5:7])
                    texts_raw['date'].append(re.sub('\n', '', news_piece[2]))                    
                    
                    # column
                    clean_column = re.sub('\n', '', news_piece[4])[3:]
                    if clean_column.strip() == '':
                        texts_raw['column'].append('NA')
                    else:
                        texts_raw['column'].append(clean_sent(clean_column))

                    # authors
                    author_str = ''
                    author_list = []
                    if news_piece[1] == '\n':
                        author_str = 'NA'
                    else:
                        author_list = re.sub('\n', '', news_piece[1]).split()
                        author_str = ' & '.join(author_list)

                    texts_raw['author'].append(author_str)
                    
                    # texts
                    title_author_list = [clean_title_list, author_list]
                    cleaned_text = text_cleaning(
                        title_author_list, news_piece[6:])
                    if cleaned_text[1] == True:
                        texts_raw['text'].append(''.join(cleaned_text[0]))
                        texts_raw['tokenized_sent'].append(tokenize_para(cleaned_text[0]))
                        texts_raw['token_para'].append([' '.join(x) for x in tokenize_para(cleaned_text[0])])
                    else:
                        # if return empty text, then this news piece is
                        # trivial, just delete it
                        del texts_raw['year'][-1]
                        del texts_raw['month'][-1]
                        del texts_raw['date'][-1]
                        del texts_raw['issue'][-1]
                        del texts_raw['column'][-1]
                        del texts_raw['title'][-1]
                        del texts_raw['author'][-1]
#                break
#            break
    return texts_raw

In [32]:
start_time = timeit.default_timer()

news_raw = loadcorpus_plus_cleaning(corpus_name)

running_time = timeit.default_timer() - start_time

if running_time < 60:
    print('\nTotal time used: {:.0f} s'.format(running_time))
else:
    print('\nTotal time used: {:.0f} min {:.0f} s'.format(running_time // 60, running_time % 60))

Current running on 1995年12月, total: 372/372
Total time used: 73 min 25 s


In [33]:
#news_raw['token_para']

In [34]:
# construct a dataframe
news_df = pd.DataFrame(news_raw)
news_df_sort = news_df.sort_values(['date', 'issue']).copy()
news_df_sort

,year,month,date,issue,column,title,author,text,tokenized_sent,token_para
193,1965,01,1965-01-01,1,NA,刘少奇主席召集最高国务会议,NA,刘主席就当前国际国内形势和工作中的一些重要问题作了讲话会议对周恩来总理在人大会议上作的“政府...,"[[主席, 当前, 国际, 国内形势, 工作, 一些, 重要, 问题, 讲话], [会议, ...","[主席 当前 国际 国内形势 工作 一些 重要 问题 讲话, 会议 周恩来 总理 人大 会议..."
836,1965,01,1965-01-01,1,NA,毛主席刘主席等领导人同五万军民联欢迎接新年,NA,首都举行盛大的拥军优属、拥政爱民新年联欢晚会人大代表政协委员也同军民联欢，全场洋溢着团结欢乐...,"[[首都, 举行, 盛大, 拥军优属, 拥政爱民, 新年, 联欢晚会], [人大代表, 政协...","[首都 举行 盛大 拥军优属 拥政爱民 新年 联欢晚会, 人大代表 政协委员 军民联欢 全场..."
161,1965,01,1965-01-01,2,NA,政协四届首次会议继续举行大会,NA,政协第三届全国委员会副主席、西藏自治区筹备委员会副主任委员帕巴拉·格列朗杰用大量事实，揭发和...,"[[政协, 第三届, 全国, 委员会, 主席, 西藏自治区筹备委员会, 主任委员, 帕巴拉,...",[政协 第三届 全国 委员会 主席 西藏自治区筹备委员会 主任委员 帕巴拉 格列朗 杰用 大...
517,1965,01,1965-01-01,2,NA,三届人大首次会议继续举行大会,NA,四十位代表作了发言或书面发言同意政府工作报告西藏代表阿沛·阿旺晋美揭露了达赖集团的叛国罪行，...,"[[四十位, 代表作, 发言, 书面发言, 同意, 政府, 工作, 报告], [西藏, 代表...","[四十位 代表作 发言 书面发言 同意 政府 工作 报告, 西藏 代表 阿沛 阿旺晋美 揭露..."
706,1965,01,1965-01-01,2,NA,三届人大首次会议 举行主席团扩大会议讨论国家领导人等候选人名单问题 三届人大首次会议的各代表...,NA,第三届全国人民代表大会第一次会议，三十日举行主席团扩大会议，讨论了即将在第三届全国人民代表...,"[[第三届, 全国人民代表大会, 第一次, 会议, 三十日, 举行, 主席团, 扩大, 会议...",[第三届 全国人民代表大会 第一次 会议 三十日 举行 主席团 扩大 会议 讨论 即将 第三...
...,...,...,...,...,...,...,...,...,...,...
399461,1995,12,1995-12-31,4_副刊,NA,喜庆丰收,周文,雷岩１９９５年对这位山东歌舞剧院的男中音（见右图）来说，是一个丰收年。在第二届聂耳、冼星海全...,"[[雷岩], [这位, 山东, 歌舞剧, 男中音, 右图, 来说, 一个, 丰收年], [第...","[雷岩, 这位 山东 歌舞剧 男中音 右图 来说 一个 丰收年, 第二届 聂耳 冼星海 全国..."
399616,1995,12,1995-12-31,4_副刊,NA,含蓄深沉,滨一,沈丹萍她１９６０年出生，是南京人，１９７６年考入扬州市歌舞团，演过歌剧、话剧、舞剧，１９８２...,"[[沈丹萍], [出生, 南京, 考入, 扬州市, 歌舞团, 歌剧, 话剧, 舞剧, 毕业,...","[沈丹萍, 出生 南京 考入 扬州市 歌舞团 歌剧 话剧 舞剧 毕业 北京电影学院 表演系 ..."
399679,1995,12,1995-12-31,4_副刊,NA,后来居上,向文,艾丽娅１９６４年出生的这位蒙古族姑娘，１９８５年毕业于北京电影学院表演系，曾先后在《女绑架者...,"[[艾丽娅], [出生, 这位, 蒙古族, 姑娘, 毕业, 北京电影学院, 表演系, 先后,...","[艾丽娅, 出生 这位 蒙古族 姑娘 毕业 北京电影学院 表演系 先后 绑架者 世界屋脊 太..."
399680,1995,12,1995-12-31,4_副刊,NA,一年硕果满枝头１９９５年艺苑桂冠选萃,NA,岁末已至，新年将临，辞旧迎新的时候，回首艺苑，发现今年又是硕果累累的一年。我国许多艺术家、艺...,"[[岁末, 已至, 新年, 将临, 辞旧迎新, 时候, 回首, 艺苑, 发现, 今年, 硕果...","[岁末 已至 新年 将临 辞旧迎新 时候 回首 艺苑 发现 今年 硕果累累 一年, 我国 许..."


In [136]:
# random check on df

news_df_sort[::700]

,year,month,date,issue,column,title,author,text,tokenized_sent
4,1988,01,1988-01-01,1_要闻,NA,国务院召开全会部署今年工作 要求各部门做好实施机构改革方案准备 李鹏强调中心任务是深化改革稳定经济,NA,在新的一年即将来临的时候，国务院于1987年12月30日召开全体会议，李鹏代总理主持了会...,"[[一年, 即将来临, 时候, 国务院, 1987, 12, 30, 召开, 全体会议, 李..."
706,1988,01,1988-01-13,3_科教·文化·体育,NA,著名翻译家戈宝权获普希金文学奖,赵伟,著名翻译家、中国社会科学院外国文学研究所研究员戈宝权今日获苏联“普希金文学奖”。今天下午，苏...,"[[著名, 翻译家, 中国社会科学院, 外国文学, 研究所, 研究员, 戈宝权, 今日, 苏..."
1393,1988,01,1988-01-26,3_科教·文化·体育,文化天地,“悬棺之谜”迷住首都观众 北京自然博物馆有一特别展览,蒋建科,在四川省珙县长达数公里的麻矿坝，有一具具悬置在几乎与地面垂直的崖壁之上的长形木棺，虽经数百年...,"[[四川省, 珙县, 数公里, 麻矿坝, 一具, 悬置, 几乎, 地面, 垂直, 崖壁, 之..."
2131,1988,02,1988-02-07,1_要闻,NA,河南举行贫困地区经济开发会 穷县竟纷纷驾小轿车直抵郑州,石建华,河南省郑州市中州宾馆前这几天小轿车一辆接一辆，出出进进，原来河南省贫困地区经济开发工作会议在...,"[[河南省, 郑州市, 中州, 宾馆, 几天, 小轿车, 一辆, 一辆, 出出进进, 原来,..."
2781,1988,02,1988-02-20,3_科教·文化·体育,NA,中国队获决赛权 十二月进军卡塔尔,吴毅宏,中国队获决赛权 十二月进军卡塔尔,"[[中国队, 决赛权, 十二月, 进军, 卡塔尔]]"
3529,1988,03,1988-03-04,1_要闻,NA,李鹏胡启立会见优秀技术人员 赞扬讲理想比贡献活动取得可喜成果,卓培荣,中共中央政治局常委、国务院代总理李鹏，中共中央政治局常委胡启立等领导同志，今天下午在中南海会...,"[[中共, 中央政治局常委, 国务院, 代总理, 李鹏, 中共, 中央政治局常委, 胡启立,..."
4180,1988,03,1988-03-18,3_科教·文化·体育,体育,全英羽毛球锦标赛开场戏 中国男子单打选手全部覆没,张明德 & 李宗扬,为期5天的第七十八届全英羽毛球锦标赛今天在这里的温布利体育中心开幕。中国男子单打、女子单打和...,"[[为期, 第七十, 八届, 全英, 羽毛球, 锦标赛, 今天, 这里, 温布利, 体育中心..."
4919,1988,04,1988-04-03,2_要闻,两会花絮,说改就改,吴友松,陆文夫代表1日在全国人大主席团会议上提出，这次会议用车量大，又采用了临时交通管理办法，给首都...,"[[陆文夫, 代表, 全国人大, 主席团, 会议, 提出, 这次, 会议, 用车, 采用, ..."
5594,1988,04,1988-04-22,2_经济,NA,三坐标测量机达到国际先进水平,晓星,三坐标测量机达到国际先进水平 由青岛前哨机械厂研制成功的ZC8645三坐标测量机，最近通过国...,"[[坐标, 测量机, 达到, 国际, 先进, 水平, 青岛, 前哨, 机械厂, 研制成功, ..."
6340,1988,05,1988-05-06,1_要闻,NA,赵紫阳会见民主德国客人时说 中国找到适合国情的建设道路,冯秀菊,中共中央总书记赵紫阳今天下午在中南海会见了由政治局委员、中央书记兼柏林专区党委第一书记京特·...,"[[中共中央, 总书记, 赵紫阳, 今天下午, 中南海, 会见, 政治局, 委员, 中央, ..."


In [137]:
# sample output the corpus to csv
if user == "Tim":
    saving_path = "/Users/timqzhang/Desktop/UChicago/MACSS_Spring_2020/content_local/rmrb/"
elif user == "Linghui":
    saving_path = "/Users/linghuiwu/uchicago/courseworks/soci40133/rmrb/"
elif user == "Minghao":
    saving_path = "D:/学习资料/研一三学期/Content Analysis/Final_Project/rmrb/"

news_df_sort.loc[news_df_sort['year'] == '1988', :].to_csv(saving_path + "rmrb_corpus.csv")

In [12]:
# Save the whole corpus for further analysis
if user == "Linghui":
    news_df_filter.to_csv("rmrb.csv")